# 各層の実装
前のセクションに引き続き、各層はクラスで実装します。  
今回は、クラスの継承を導入し重複部分を共通化します。  
各層のベースとなるクラスを実装し、このクラスを継承して中間層と出力層を実装します。 

## ●継承元のクラス
継承元のクラス、`BaseLayer`は以下のように実装します。

In [ ]:
# -- 各層の継承元 --
class BaseLayer:
    def __init__(self, n_upper, n):
        self.w = wb_width * np.random.randn(n_upper, n)  # 重み（行列）
        self.b = wb_width * np.random.randn(n)  # バイアス（ベクトル）

    def update(self, eta):
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

`__init__`メソッドと`update`メソッドは各層で共通なので、`BaseLayer`クラスに実装します。  
`__init__`では初期設定が行われ、`update`では確率的勾配降下法に基づき重みとバイアスが更新されます。

## ●中間層のクラス
`BaseLayer`クラスを継承して、中間層を表すクラス`MiddleLayer`を実装します。

In [ ]:
# -- 中間層 --
class MiddleLayer(BaseLayer):
    def forward(self, x):
        self.x = x
        self.u = np.dot(x, self.w) + self.b
        self.y = np.where(self.u <= 0, 0, self.u) # ReLU
    
    def backward(self, grad_y):
        delta = grad_y * np.where(self.u <= 0, 0, 1)  # ReLUの微分

        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        
        self.grad_x = np.dot(delta, self.w.T) 

順伝播用の`forward`メソッドと、逆伝播用の`backward`メソッドの実装はほぼ前のセクションと同じですが、活性化関数にReLUを使用している点が異なります。

`np.where(self.u <= 0, 0, 1)`はReLUの微分形です。  
継承を利用しているので、以前と比べてコードがスリムになっています。  

## ●出力層のクラス
同様に、出力層である`OutputLayer`も`BaseLayer`を継承します。

In [ ]:
# -- 出力層 --
class OutputLayer(BaseLayer):     
    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = np.exp(u)/np.sum(np.exp(u), axis=1, keepdims=True)  # ソフトマックス関数

    def backward(self, t):
        delta = self.y - t
        
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        
        self.grad_x = np.dot(delta, self.w.T) 

`forward`メソッドと`backward`メソッドの実装は、前のセクションと同じです。

## ●ニューラルネットワークの実装
中間層と出力層のクラスを用いて、ニューラルネットワークを構築します。  
今回はニューラルネットワークに関する各処理を関数の形で実装します。

In [ ]:
# -- 各層の初期化 --
middle_layer_1 = MiddleLayer(n_in, n_mid)
middle_layer_2 = MiddleLayer(n_mid, n_mid)
output_layer = OutputLayer(n_mid, n_out)

# -- 順伝播 --
def forward_propagation(x):
    middle_layer_1.forward(x)
    middle_layer_2.forward(middle_layer_1.y)
    output_layer.forward(middle_layer_2.y)

# -- 逆伝播 --
def backpropagation(t):
    output_layer.backward(t)
    middle_layer_2.backward(output_layer.grad_x)
    middle_layer_1.backward(middle_layer_2.grad_x)

# -- 重みとバイアスの更新 --
def uppdate_wb():
    middle_layer_1.update(eta)
    middle_layer_2.update(eta)
    output_layer.update(eta)

# -- 誤差を計算 --
def get_error(t, batch_size):
    return -np.sum(t * np.log(output_layer.y + 1e-7)) / batch_size  # 交差エントロピー誤差

今回のニューラルネットワークには中間層が2つ、出力層が1つあります。  
各層のインスタンスを生成した上で、これらを用いて順伝播と逆伝播の関数（```forward_propagation```、```backpropagation```）を実装します。  
また、全ての重みとバイアスを更新する関数（```uppdate_wb```）も実装しておきます。  

誤差を計算する関数（```get_error```）も実装しますが、今回は分類問題なので交差エントロピー誤差を計算します。  
ミニバッチ学習なので、誤差の総和をバッチサイズで割ります。